In [1]:
# imports
import pandas as pd
from datetime import datetime
from LanguageDetection import LanguageDetection as lang
import regex as re
import langid

In [2]:
# load data and drop obsolete columns
df_twitter_old = pd.read_csv('twitter_res/twitter_sentiment_data.csv')
df_twitter_old.drop(columns=['sentiment'], inplace=True)
df_twitter_old.rename(columns={'message':'tweet', 'tweetid':'tweetsid'}, inplace=True)

df_twitter_new = pd.read_csv('twitter_res/climate_change_tweets.csv')
df_twitter_new.drop(columns=['hashtag'], inplace=True)

In [3]:
# concat
df_twitter = pd.concat([df_twitter_old, df_twitter_new])
df_twitter = df_twitter.reset_index()
df_twitter = df_twitter.head(500)

In [4]:
df_twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     500 non-null    int64 
 1   tweet     500 non-null    object
 2   tweetsid  500 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 11.8+ KB


**Remove URLs and users marked with @**

In [5]:
def remove_url(tweet):
    return re.sub(r'https?://\S+|www\.\S+', r"", tweet)


def remove_user(tweet):
    return re.sub(r'@[^\s]+', r"", tweet)

# Remove @ and urls
df_twitter["tweet"] = df_twitter["tweet"].apply((lambda tweet: remove_url(tweet)))
df_twitter["tweet"] = df_twitter["tweet"].apply((lambda tweet: remove_user(tweet)))

**Drop rows, where the tweet is not in English**

In [6]:
def detect_language(tweet):
   # language = lang.LanguageDetection().LanguageDetect(tweet)
   # language = LanguageIdentifier.from_modelstring(model, norm_probs=True)
    pred = langid.classify(tweet)
    if pred[0] != "en":
        return False
    else:
        return True
# Filtering language
to_drop = []
df_twitter['tweet'] = df_twitter['tweet'].apply(lambda tweet: tweet if (detect_language(tweet)) else to_drop.append(
    df_twitter[df_twitter["tweet"] == tweet].index[0]))
df_twitter = df_twitter.drop(to_drop)

**Add column with date encoding the ID from the tweets**
Source: https://github.com/oduwsdl/tweetedat


In [7]:
def get_timestamp(id):
    time = datetime.utcfromtimestamp(((id >> 22) + 1288834974657) / 1000)
    return time.strftime("%d-%m-%Y")

df_twitter['tweetsid'] = df_twitter['tweetsid'].apply(lambda id: get_timestamp(id))
df_twitter = df_twitter.drop(columns="tweetsid")


In [8]:
print(df_twitter)

     index                                              tweet
0        0   climate change is an interesting hustle as it...
1        1  RT  Watch #BeforeTheFlood right here, as  trav...
2        2  Fabulous! Leonardo #DiCaprio's film on #climat...
3        3  RT  Just watched this amazing documentary by l...
4        4  RT  Pranita Biswasi, a Lutheran from Odisha, g...
..     ...                                                ...
495    495  Watch Leo DiCaprio talk #climate change with E...
496    496  RT  The effect of climate change on the food c...
497    497  RT  When youre enjoying the warm weather but y...
498    498  RT  Appetite for oil and gas will continue to ...
499    499  RT  Via  Cartoon: If the media covered climate...

[466 rows x 2 columns]
